In [1]:
#env: equake_env (Python 3.9)
import pandas as pd
import sqlite3
import datetime
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

from sqlalchemy import create_engine

In [2]:
#connection to a PostgreSQL local database
# ===========================================

# user_name_var = input('What is the username of your database? ')
host_var = input('What is the host address of your database?')
pw = input("Enter the database password: ")
# db_name = input("Enter the name of the database: ")

string_output = "postgresql://postgres"+':'+pw+'@'+host_var+":5432"+'/earthquake_db'
engine = create_engine(string_output)
connection = engine.connect()
print("Connected to database...")

Connected to database...


In [3]:
#create and connect to a sqlite3 db
path = "../Database/db/earthquake_db"
conn = sqlite3.connect(path)

Adding 1 month of data

In [4]:
page_url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"

df = pd.read_csv(page_url)
df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-06-28T18:11:21.860Z,33.512333,-116.548833,12.96,0.72,ml,31.0,43.0,0.047450,0.19,...,2022-06-28T18:15:04.334Z,"12km ESE of Anza, CA",earthquake,0.29,0.52,0.11400,21.0,automatic,ci,ci
1,2022-06-28T18:11:14.790Z,33.188667,-116.002500,6.57,0.98,ml,24.0,72.0,0.025690,0.30,...,2022-06-28T18:14:53.782Z,"13km SSW of Salton City, CA",earthquake,0.66,1.05,0.15900,16.0,automatic,ci,ci
2,2022-06-28T17:39:47.560Z,38.825333,-122.798668,2.96,0.46,md,9.0,179.0,0.009117,0.01,...,2022-06-28T18:21:16.894Z,"6km NW of The Geysers, CA",earthquake,0.55,0.73,0.39000,2.0,automatic,nc,nc
3,2022-06-28T17:36:15.411Z,62.017600,-151.067700,82.60,1.90,ml,NaN,NaN,NaN,0.49,...,2022-06-28T17:38:45.435Z,"17 km E of Skwentna, Alaska",earthquake,NaN,1.00,NaN,NaN,automatic,ak,ak
4,2022-06-28T17:27:11.303Z,60.605800,-147.770000,18.40,2.50,ml,NaN,NaN,NaN,0.51,...,2022-06-28T18:18:54.040Z,"53 km ESE of Whittier, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9071,2022-05-29T18:46:07.881Z,62.765700,-149.147900,64.00,1.30,ml,NaN,NaN,NaN,1.14,...,2022-05-29T18:47:50.375Z,"60 km NE of Chase, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
9072,2022-05-29T18:45:49.520Z,35.520167,-117.433000,5.65,1.77,ml,28.0,62.0,0.056060,0.14,...,2022-06-01T15:14:27.920Z,"24km NE of Johannesburg, CA",earthquake,0.19,0.42,0.07800,12.0,reviewed,ci,ci
9073,2022-05-29T18:39:05.860Z,39.259667,-123.484667,8.82,1.76,md,21.0,58.0,0.127000,0.09,...,2022-06-02T01:34:11.951Z,"20km SSW of Willits, CA",earthquake,0.18,0.60,0.12300,16.0,reviewed,nc,nc
9074,2022-05-29T18:35:31.810Z,38.835000,-122.810500,2.13,1.18,md,42.0,38.0,0.011590,0.03,...,2022-06-09T01:08:14.796Z,"8km W of Cobb, CA",earthquake,0.14,0.27,0.07800,12.0,reviewed,nc,nc


In [5]:
#creates/replaces database to add new data to json file
#=============
#PostgreSQL
df.to_sql('earthquake_update', engine, if_exists='replace', index = False)
#=============
#SQLite
cur = conn.cursor()
df.to_sql('earthquake_update', conn, if_exists='replace', index=False)

9076

Load from database to update with new rows and remove duplicate rows

In [6]:
#Dataframe from postgresql
model_df = pd.read_sql_table(
    "earthquake_raw",
    con=engine
)
model_df.shape

(64883, 22)

In [7]:
#dataframe from SQLite
model_df2 = pd.read_sql_query("SELECT * FROM earthquake_raw", conn)
model_df2.shape

(64883, 22)

In [8]:
#concatenate new data df with data read from postgres db table
model_df = pd.concat([model_df,df], verify_integrity=True, ignore_index=True)
model_df.shape

(73959, 22)

In [9]:
#concatenate new data df with data read from sqlite db table
model_df2 = pd.concat([model_df2,df], verify_integrity=True, ignore_index=True)
model_df2.shape

(73959, 22)

In [10]:
#check for duplicates in postgres data
boolean = model_df.duplicated(subset=['id']).any()
print(boolean)

True


In [11]:
#check for duplicates in sqlite data
boolean = model_df2.duplicated(subset=['id']).any()
print(boolean)

True


In [12]:
#drop duplicates in postgres data
model_df.drop_duplicates(subset=['id'], inplace=True)
print(model_df)

                           time   latitude   longitude   depth   mag magType  \
0      2022-01-06T01:42:39.855Z  37.384000 -117.107500    1.10  1.30      ml   
1      2022-01-06T01:31:33.698Z  38.181000 -117.818000    7.30  1.40      ml   
2      2022-01-06T01:24:20.083Z  61.340700 -149.885200   30.00  1.50      ml   
3      2022-01-06T01:15:15.851Z  62.870300 -150.762500  111.30  2.10      ml   
4      2022-01-06T01:10:44.629Z  64.014500 -148.765700    1.90  1.50      ml   
...                         ...        ...         ...     ...   ...     ...   
65160  2022-06-27T18:58:42.890Z  32.676333 -115.442833   16.13  1.12      ml   
65378  2022-06-26T23:55:51.750Z  35.989333 -117.432333    2.80  0.76      ml   
65546  2022-06-26T11:16:17.540Z  36.101500 -117.859833    2.55  0.76      ml   
65635  2022-06-26T05:22:29.360Z  36.104000 -117.869500    3.19  0.78      ml   
65764  2022-06-25T16:56:09.410Z  35.829167 -117.682667    8.17  0.63      ml   

        nst     gap     dmin     rms  .

In [13]:
#drop duplicates in sqlite data
model_df2.drop_duplicates(subset=['id'], inplace=True)
print(model_df2)

                           time   latitude   longitude   depth   mag magType  \
0      2022-01-06T01:42:39.855Z  37.384000 -117.107500    1.10  1.30      ml   
1      2022-01-06T01:31:33.698Z  38.181000 -117.818000    7.30  1.40      ml   
2      2022-01-06T01:24:20.083Z  61.340700 -149.885200   30.00  1.50      ml   
3      2022-01-06T01:15:15.851Z  62.870300 -150.762500  111.30  2.10      ml   
4      2022-01-06T01:10:44.629Z  64.014500 -148.765700    1.90  1.50      ml   
...                         ...        ...         ...     ...   ...     ...   
65160  2022-06-27T18:58:42.890Z  32.676333 -115.442833   16.13  1.12      ml   
65378  2022-06-26T23:55:51.750Z  35.989333 -117.432333    2.80  0.76      ml   
65546  2022-06-26T11:16:17.540Z  36.101500 -117.859833    2.55  0.76      ml   
65635  2022-06-26T05:22:29.360Z  36.104000 -117.869500    3.19  0.78      ml   
65764  2022-06-25T16:56:09.410Z  35.829167 -117.682667    8.17  0.63      ml   

        nst     gap     dmin     rms  .

In [14]:
#replace the table in postgres with one without duplicates
model_df.to_sql('earthquake_raw', engine, if_exists='replace', index = False)

891

In [15]:
#replace the table in sqlite with one without duplicates
model_df2.to_sql('earthquake_raw', conn, if_exists='replace', index = False)

64891

In [16]:
#copy data as csv as backup method
model_df.to_csv("../Database/csv/earthquake_raw.csv",index=False)

In [17]:
# Close connection to SQLite database
conn.close()
connection.close()